In [60]:
# !pip install finance-datareader
# !pip install pycaret

In [61]:
import pandas as pd
import numpy as np
import os
from pycaret.regression import *
import FinanceDataReader as fdr
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from tqdm import tqdm

## Get Stock List

In [62]:

path = ''
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'
stock_data = '20210104_20211119_data_interpolate.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_data = pd.read_csv(os.path.join(path,stock_data))
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))


In [63]:
# import lightgbm as lgb
# model = LinearRegression()
# for code in tqdm(stock_list['종목코드'].values):
#   data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
#   data = pd.merge(stock_data, data,on='Date',how='left').set_index('Date')
#   X = data[fratures]
#   y = data['Close']
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [64]:
# Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
# code = '005930'
# data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
# data = pd.merge(Business_days, data, how = 'outer')
# data['weekday'] = data.Date.apply(lambda x : x.weekday())
# data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
# data.Close = data.Close.interpolate()


In [65]:
# merge_stock_data.to_datetime()
# data = pd.merge(data, merge_stock_data, on='Date')

In [66]:
start_date = '20210104'
end_date = '20211119'

sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()

sample = pd.merge(stock_data,sample,on='Date',how='left')

train = sample[sample['Date'] < '2021-11-01']
test =  sample[sample['Date'] <= '2021-11-05']
test =  test[test['Date'] >= '2021-11-01']
test = pd.DataFrame(test['Date']).reset_index().drop('index',axis=1)

In [67]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 0 to 214
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           215 non-null    datetime64[ns]
 1   exchange_rate  215 non-null    float64       
 2   kospi          215 non-null    float64       
 3   kosdaq         215 non-null    float64       
 4   kospi100       215 non-null    float64       
 5   dow            215 non-null    float64       
 6   nasdaq         215 non-null    float64       
 7   sp500          215 non-null    float64       
 8   미국채10년-2년      215 non-null    float64       
 9   BTC            215 non-null    float64       
 10  Gold           215 non-null    float64       
 11  Oil            215 non-null    float64       
 12  US10Y          215 non-null    float64       
 13  US2Y           215 non-null    float64       
 14  Close          204 non-null    float64       
dtypes: datetime64[ns](1), f

In [68]:

features = ['Date','kospi','kosdaq','kospi100','dow','nasdaq','sp500','exchange_rate','미국채10년-2년','BTC', 'US2Y','US10Y','Gold','Oil']
train = train[features]

stock_pred_dict = {} 
for i in range(len(features)-1):
  model=setup(
      data=train,
      target = features[i+1],
      ignore_features = features[i+2:],
      log_experiment = True,
      normalize = True,
      silent=True
    )
  tuned_model = tune_model(compare_models(sort ='MAPE', n_select=1))
  tuned_model_ = finalize_model(tuned_model)
  pred = predict_model(tuned_model_, test)
  test[features[i+1]] = pred['Label']


,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.9570,1.3614,1.1668,0.9180,0.0258,0.0214
1,0.8931,1.7718,1.3311,0.9504,0.0311,0.0211
2,1.0833,2.5151,1.5859,0.9338,0.0354,0.0252
3,0.7208,0.9219,0.9602,0.9692,0.0207,0.0160
4,0.5455,0.6187,0.7866,0.9801,0.0179,0.0127
5,1.0614,1.7742,1.3320,0.9334,0.0285,0.0234
6,0.9910,1.3095,1.1443,0.9414,0.0245,0.0219
7,0.9363,1.4049,1.1853,0.9520,0.0236,0.0189
8,1.1191,1.5377,1.2400,0.9283,0.0274,0.0253
9,0.5719,0.4818,0.6941,0.9834,0.0148,0.0125


In [ ]:

for code in tqdm(stock_list['종목코드'].values):
  code_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  train['Close'] = code_data['Close']
  model=setup(
        data=train,
        target = 'Close',
        log_experiment = True,
        normalize = True,
        silent=True
      )
  tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
  tuned_model_ = finalize_model(tuned_model)
  pred = predict_model(tuned_model_, test)
  stock_pred_dict[code] = pred['Label']


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,73.6677,9.629972e+03,95.9654,0.8797,0.0289,0.0221,0.408
gbr,Gradient Boosting Regressor,78.9731,1.127485e+04,103.0016,0.8580,0.0308,0.0235,0.074
rf,Random Forest Regressor,83.2717,1.294071e+04,109.2044,0.8347,0.0328,0.0249,0.463
knn,K Neighbors Regressor,87.4200,1.409183e+04,116.1060,0.8333,0.0362,0.0267,0.063
ada,AdaBoost Regressor,89.3994,1.416409e+04,115.5364,0.8214,0.0353,0.0270,0.086
huber,Huber Regressor,92.6020,1.595136e+04,123.9856,0.8004,0.0383,0.0281,0.042
lightgbm,Light Gradient Boosting Machine,95.6965,1.567225e+04,123.4630,0.8105,0.0371,0.0287,0.028
lr,Linear Regression,98.6472,1.681127e+04,126.7342,0.7877,0.0387,0.0297,0.013
lasso,Lasso Regression,99.2976,1.622813e+04,124.7886,0.7915,0.0382,0.0299,0.018
ridge,Ridge Regression,100.6174,1.662953e+04,126.3407,0.7865,0.0388,0.0304,0.014


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:09:19
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   21.9s


In [ ]:
test

In [ ]:
train

- 예측

- 실제 Public 값